# Cargo Airplane: Implementation in Python (Optional)

```{custom_download_link} ./environment_MUDE_opt_book.yml
:text: "Environment"
:replace_default: "False"
```

```{custom_download_link} ./airplane_original_stripped.ipynb
:replace_default: "True"
```

```{note}
Gurobi cannot be loaded in this online book, so download this notebook to work on it with Gurobi locally installed. Instruction on how to do that are in the `README.md` and `PA_2_4_A_gurobilicious.ipynb` of week 2.4.
```

```{admonition} MUDE Exam Information
:class: tip, dropdown
For the exam, you are expected to have a clear understanding behind the Simplex method and how to do it. you will not need to do the calculations by hand in the exam or on Friday’s assignment.
```



In [13]:
import gurobipy as gp
import numpy as np
import pandas as pd

model = gp.Model()

In [14]:
# cargo limits
cargo_type, cargo_weight, cargo_volume, cargo_profit = gp.multidict({
    'Type1': [18, 180, 590],
    'Type2': [14, 250, 580],
    'Type3': [20, 300, 580],
    'Type4': [12, 80,  600]
})

# compartment limits
compartment_type, compartment_weight, compartment_volume = gp. multidict({
    'Front': [8,  12000],
    'Center':[12, 10000],
    'Rear':  [14, 6000]
})

In [15]:
## decision variables
x = model.addVars(len(cargo_type),len(compartment_type), vtype = gp.GRB.CONTINUOUS, name = 'x')
model.update()

In [16]:
# set objectives
objective = gp.quicksum(cargo_profit[cargo_type[i]] * x[i, j]
                               for j in range(len(compartment_type))
                               for i in range(len(cargo_type)))

model.setObjective(objective, gp.GRB.MAXIMIZE)

In [ ]:
## constraints
# 1. cargo type weight limits
model.addConstrs((gp.quicksum(x[i, j] for j in range(len(compartment_type))) <= cargo_weight[cargo_type[i]]
                  for i in range(len(cargo_type))), name ='Max available cargo to carry')

In [ ]:
# 2. compartment weight limits
model.addConstrs((gp.quicksum(x[i, j] for i in range(len(cargo_type))) <= compartment_weight[compartment_type[j]]
                  for j in range(len(compartment_type))), name ='Weight limits on the compartments')

In [ ]:
# 3. compartment volume limitations
model.addConstrs((gp.quicksum(x[i, j]*cargo_volume[cargo_type[i]] for i in range(len(cargo_type))) <= compartment_volume[compartment_type[j]]
                  for j in range(len(compartment_type))), name ='Volume limits on the compartments')

In [20]:
# Complete model
model.update()
model.display()

Maximize
590.0 x[0,0] + 590.0 x[0,1] + 590.0 x[0,2] + 580.0 x[1,0] + 580.0 x[1,1]
+ 580.0 x[1,2] + 580.0 x[2,0] + 580.0 x[2,1] + 580.0 x[2,2] + 600.0 x[3,0]
+ 600.0 x[3,1] + 600.0 x[3,2]
Subject To
  Max available cargo to carry[0]: x[0,0] + x[0,1] + x[0,2] <= 18
  Max available cargo to carry[1]: x[1,0] + x[1,1] + x[1,2] <= 14
  Max available cargo to carry[2]: x[2,0] + x[2,1] + x[2,2] <= 20
  Max available cargo to carry[3]: x[3,0] + x[3,1] + x[3,2] <= 12
  Weight limits on the compartments[0]: x[0,0] + x[1,0] + x[2,0] + x[3,0] <= 8
  Weight limits on the compartments[1]: x[0,1] + x[1,1] + x[2,1] + x[3,1] <= 12
  Weight limits on the compartments[2]: x[0,2] + x[1,2] + x[2,2] + x[3,2] <= 14
Volume limits on the compartments[0]: 180.0 x[0,0] + 250.0 x[1,0] + 300.0 x[2,0] + 80.0
 x[3,0] <= 12000
Volume limits on the compartments[1]: 180.0 x[0,1] + 250.0 x[1,1] + 300.0 x[2,1] + 80.0
 x[3,1] <= 10000
Volume limits on the compartments[2]: 180.0 x[0,2] + 250.0 x[1,2] + 300.0 x[2,2] + 80.0
 

C:\Users\tomvanwoudenbe\AppData\Local\Temp\ipykernel_7320\1502344741.py:3: DeprecationWarning: Model.display() is deprecated
  model.display()


In [ ]:
# Solve
model.optimize()